In [ ]:
import json
from collections import defaultdict
from columnar import columnar
from statistics import mean, stdev
import matplotlib.pyplot as plt

In [ ]:
sets = json.load(open("data/all_sets.json", "r"))

In [ ]:
lookup_names = defaultdict(set)
lookup_name = {}
"""set id, player 1, player 2, winner"""
sets_formated = []

for set_ in sets:
    P = []

    lookup_entrant_player = {}
    for slot in set_["slots"]:
        entrant = slot["entrant"]
        participants = entrant["participants"]
        if len(participants) > 1:
            print("More than one participant")
            continue

        participant = participants[0]
        entrant_id = entrant["id"]
        player_id = participant["player"]["id"]
        entrant_name = entrant["name"]

        lookup_entrant_player[entrant_id] = player_id
        lookup_name[player_id] = entrant_name
        lookup_names[player_id].add(entrant_name)
        P.append(player_id)

    W = lookup_entrant_player[set_["winnerId"]]

    sets_formated.append((set_["id"], *P, W))

"""order the sets by id to have them chronologically"""
sets_formated.sort(key=lambda l: l[0])

In [ ]:
def calculate_expected_outcome(rating1, rating2):
    """
    Calculate the expected outcome for Player 1 based on their ratings.
    """
    return 1 / (1 + 10**((rating2 - rating1) / 400))

def update_ratings(rating1, rating2, actual_outcome, k):
    """
    Update the ratings of two players based on the match outcome.
    """
    expected_outcome = calculate_expected_outcome(rating1, rating2)

    new_rating1 = rating1 + k * (actual_outcome - expected_outcome)
    new_rating2 = rating2 + k * ((1 - actual_outcome) - (1 - expected_outcome))

    return new_rating1, new_rating2, expected_outcome

def compute_elo(sets_, initial_elo, k):
    players_stats = {}

    for id_ in lookup_names:
        players_stats[id_] = {
            "elo": initial_elo,
            "wins": 0,
            "loses": 0,
        }

    errors = []
    for set_id, player1, player2, winner in sets_:
        player1_rating = players_stats[player1]["elo"]
        player2_rating = players_stats[player2]["elo"]
        if winner == player1:
            players_stats[player1]["wins"] += 1
            players_stats[player2]["loses"] += 1
            actual_outcome = 1
        else:
            players_stats[player2]["wins"] += 1
            players_stats[player1]["loses"] += 1
            actual_outcome = 0

        new_player1_rating, new_player2_rating, expected_outcome = update_ratings(player1_rating, player2_rating, actual_outcome, k)
        players_stats[player1]["elo"] = new_player1_rating
        players_stats[player2]["elo"] = new_player2_rating

        error = (expected_outcome - actual_outcome) ** 2
        errors.append(error)

    mean_error = mean(errors)

    return players_stats, mean_error

In [ ]:
X = list(range(20, 300))
Y = []
for i in X:
    players_stats, mean_error = compute_elo(sets_formated, initial_elo=1500, k=i)
    Y.append(mean_error)

minX = min(X, key=lambda i: Y[X.index(i)])
print(minX)

plt.plot(X,Y)
plt.xlabel("K")
plt.ylabel("Error")
plt.show()

In [ ]:
players_stats, mean_error = compute_elo(sets_formated, initial_elo=1500, k=179)

players_stats_list = list(players_stats.items())
players_stats_list.sort(key=lambda player_stats: -player_stats[-1]["elo"])

pure_elos = [e[-1]["elo"] for e in players_stats_list]

print("Mean Elo (Elo initial)", mean(pure_elos))
print("Std Elo", stdev(pure_elos))

data = []

for rank, (id_, player_stats) in enumerate(players_stats_list):
    data.append([rank+1, lookup_name[id_], f"{player_stats['elo']:.1f}", player_stats["wins"], player_stats["loses"], player_stats["wins"] + player_stats["loses"]])

table = columnar(data, headers=['rank', 'pseudo', 'elo', 'W', "L", "S"], justify=['r', 'l','l', 'r', 'r', 'r', 'r'], no_borders=True)
print(table)